# Inputs

In [1]:
splitPartCount = 5
splitSeed = 27
hoursPerWindow = 1

`functionize-notebook` has modified this notebook during execution. The following variables have been injected:

- hoursPerWindow: 3


# Preproccess

## read data

In [3]:
from utils.class_patient import Patients


patients = Patients.loadPatients()
len(patients)

1213

## remove missing

In [4]:
# fill measures whose null represent false value

from constants import NULLABLE_MEASURES


nullableMeasures = NULLABLE_MEASURES

for measureName in nullableMeasures:
    patients.fillMissingMeasureValue(measureName, 0)

In [5]:
# remove measures with less than 80% of data

measures = patients.getMeasures()

for measure, count in measures.items():
    if count < len(patients) * 80 / 100:
        patients.removeMeasures([measure])
        print(measure, count)

pco2 917
ph 954
po2 917
albumin 406
hba1c 326
lymphocyte 446
height 415
urine-ketone 294
crp 19


In [6]:
# remove patients with less than 80% of data

patients.removePatientByMissingFeatures()
len(patients)

1206

## split patients

In [7]:
splitedPatients = patients.split(splitPartCount, splitSeed)

len(splitedPatients[0])

242

In [8]:
from pandas import Timedelta


splitedPatients[0].getMeasuresBetween(Timedelta(-6), Timedelta(1))

,subject_id,hadm_id,stay_id,akd,ag,age,bg,bicarbonate,bun,calcium,...,rr,saps2,sbp,scr,sodium,sofa,use_NaHCO3,uti,wbc,weight
0,14866589,20066152,33060916,False,NaN,37,NaN,NaN,NaN,NaN,...,NaN,22,NaN,NaN,133.0,2,0.0,0,NaN,NaN
0,17922352,20070680,39895564,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,26,NaN,NaN,145.0,5,0.0,0,NaN,NaN
0,14849360,20152551,32817197,False,NaN,59,NaN,NaN,NaN,NaN,...,NaN,31,NaN,NaN,145.0,7,0.0,0,NaN,NaN
0,16815664,20240670,37751533,False,NaN,42,NaN,NaN,NaN,NaN,...,NaN,27,NaN,NaN,138.0,2,NaN,0,NaN,43.00
0,16792064,20244311,33026932,False,NaN,46,NaN,NaN,NaN,NaN,...,NaN,19,NaN,NaN,131.0,0,0.0,0,NaN,68.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,15517352,29801212,32814591,False,NaN,20,NaN,NaN,NaN,NaN,...,NaN,30,NaN,NaN,139.0,4,NaN,0,NaN,NaN
0,10383045,29899941,39755932,False,NaN,58,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,1,0.0,0,NaN,59.40
0,15679298,29905973,39834726,False,NaN,50,NaN,NaN,NaN,NaN,...,NaN,32,NaN,NaN,134.0,3,0.0,0,NaN,47.40
0,10912213,29911812,34040470,False,NaN,47,NaN,NaN,NaN,NaN,...,NaN,23,NaN,NaN,136.0,2,0.0,0,NaN,89.05


In [9]:
splitedPatients = patients.split(splitPartCount, splitSeed)


def trainTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        yield trainPatients, testPatients


def trainValTest():
    for i in range(splitedPatients.__len__()):
        testPatients = splitedPatients[i]

        trainPatientsList = splitedPatients[:i] + splitedPatients[i + 1 :]
        trainPatients = Patients(patients=[])
        for trainPatientsElem in trainPatientsList:
            trainPatients += trainPatientsElem

        *trainPatients, valPatients = trainPatients.split(5, 27)
        tmpPatients = Patients(patients=[])
        for trainPatientsElem in trainPatients:
            tmpPatients += trainPatientsElem
        trainPatients = tmpPatients

        yield trainPatients, valPatients, testPatients

In [10]:
for trainPatients, testPatients in trainTest():
    print(len(trainPatients.patientList), len(testPatients.patientList))

964 242
965 241
965 241
965 241
965 241


In [11]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


dataset = []
for i, (trainPatients, testPatients) in enumerate(trainTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )
    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]

    dataset.append((npTrainX, trainY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [12]:
import pickle


(TEMP_PATH / f"train-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(dataset)
)

15520655

In [13]:
from constants import CATEGORICAL_MEASURES, TEMP_PATH
from utils.prepare_data import patientsToNumpy


datavalset = []
for i, (trainPatients, valPatients, testPatients) in enumerate(trainValTest()):
    # trainPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTrainX, categoryEncoder, numericEncoder, oulier, columns = patientsToNumpy(
        trainPatients, hoursPerWindow, CATEGORICAL_MEASURES
    )

    # testPatients.fillMissingMeasureValue(CATEGORICAL_MEASURES, 0)
    npTestX, *_ = patientsToNumpy(
        testPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    npValX, *_ = patientsToNumpy(
        valPatients,
        hoursPerWindow,
        CATEGORICAL_MEASURES,
        columns,
        categoryEncoder,
        numericEncoder,
        oulier,
    )

    trainY = [p.akdPositive for p in trainPatients]
    testY = [p.akdPositive for p in testPatients]
    valY = [p.akdPositive for p in valPatients]

    datavalset.append((npTrainX, trainY, npValX, valY, npTestX, testY))

    print(f"Dataset {i} created")
    pass

Dataset 0 created


Dataset 1 created


Dataset 2 created


Dataset 3 created


Dataset 4 created


In [14]:
import pickle

(TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").write_bytes(
    pickle.dumps(datavalset)
)

15289353

# LSTM

In [15]:
import pickle
from constants import TEMP_PATH

# load if not exist dataset
if "dataset" not in locals():
    dataset = pickle.loads(
        (TEMP_PATH / f"dataset-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )
   

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["AUC", "accuracy", "precision", "recall"])
    return model

2024-06-07 23:04:51.068238: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-07 23:04:51.087628: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-06-07 23:04:51.508802: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npTestX, testY) in enumerate(dataset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        class_weight=weight,
        callbacks=[early_stopping],
    )

    testY = np.array(testY)
    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


2024-06-07 23:04:51.715625: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-07 23:04:51.730981: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/tu/codepy/hust.year2023.PredictingRiskDiabeticKetoacidosis-associatedKidneyInjury/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer usi

 1/25 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - AUC: 0.5853 - accuracy: 0.6562 - loss: 0.7090 - precision: 0.6000 - recall: 0.6429

21/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.6354 - accuracy: 0.6205 - loss: 0.6660 - precision: 0.5111 - recall: 0.5336

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - AUC: 0.6492 - accuracy: 0.6295 - loss: 0.6616 - precision: 0.5242 - recall: 0.5423 - val_AUC: 0.7422 - val_accuracy: 0.6788 - val_loss: 0.5892 - val_precision: 0.5500 - val_recall: 0.7639


...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.8902 - accuracy: 0.8750 - loss: 0.4179 - precision: 0.9333 - recall: 0.8235

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9333 - accuracy: 0.8976 - loss: 0.3227 - precision: 0.9062 - recall: 0.8450 - val_AUC: 0.6443 - val_accuracy: 0.6632 - val_loss: 0.8584 - val_precision: 0.5538 - val_recall: 0.5000


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.7715 - accuracy: 0.7188 - loss: 0.5650 - precision: 0.7059 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.7814 - accuracy: 0.7020 - loss: 0.5763 - precision: 0.6124 - recall: 0.7682


Loses: [0.5939823985099792, 0.5542413592338562, 0.47507575154304504, 0.5974304676055908, 0.6042492985725403] 0.5649958550930023 0.048237621866602756
AUCs: [0.7428213357925415, 0.792227566242218, 0.8599855899810791, 0.7574982047080994, 0.7593006491661072] 0.782366669178009 0.042038671135973316
Accuracies: [0.702479362487793, 0.7468879818916321, 0.7842323780059814, 0.6763485670089722, 0.6846473217010498] 0.7189191222190857 0.04075574235499843
Precisions: [0.6017699241638184, 0.6320000290870667, 0.7009345889091492, 0.5858585834503174, 0.577235758304596] 0.6195597767829895 0.04476648691683155
Recals: [0.7157894968986511, 0.8404255509376526, 0.7894737124443054, 0.6105263233184814, 0.7473683953285217] 0.7407166957855225 0.07737473218496611


## Static validate 

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


def createModel(timeSteps, features):
    model = Sequential()
    model.add(LSTM(64, input_shape=(timeSteps, features), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(32))
    model.add(Dropout(0.5))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))

    optimizer = Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["AUC", "accuracy", "precision", "recall"],
    )
    return model

In [19]:
# crossfold
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

if "datavalset" not in locals():
    datavalset = pickle.loads(
        (TEMP_PATH / f"train-val-test-set-{splitPartCount}-{splitSeed}-{hoursPerWindow}.pkl").read_bytes()
    )

loses = []
aucs = []
accuracies = []
precisions = []
recals = []
for i, (npTrainX, trainY, npValX, valY, npTestX, testY) in enumerate(datavalset):
    npTrainX = np.nan_to_num(npTrainX, nan=0)
    npValX = np.nan_to_num(npValX, nan=0)
    npTestX = np.nan_to_num(npTestX, nan=0)

    model = createModel(npTrainX.shape[1], npTrainX.shape[2])

    neg, pos = np.bincount(trainY)
    weight0 = (1 / neg) * (len(trainY)) / 2.0
    weight1 = (1 / pos) * (len(trainY)) / 2.0
    weight = {0: weight0, 1: weight1}

    early_stopping = EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )

    trainY = np.array(trainY)
    valY = np.array(valY)
    testY = np.array(testY)

    model.fit(
        npTrainX,
        trainY,
        epochs=50,
        batch_size=32,
        validation_data=(npValX, valY),
        class_weight=weight,
        callbacks=[early_stopping],
    )

    loss, auc, accuracy, precison, recal = model.evaluate(npTestX, testY)
    loses.append(loss)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions.append(precison)
    recals.append(recal)

    pass

print("Loses:", loses, np.mean(loses), np.std(loses))
print("AUCs:", aucs, np.mean(aucs), np.std(aucs))
print("Accuracies:", accuracies, np.mean(accuracies), np.std(accuracies))
print("Precisions:", precisions, np.mean(precisions), np.std(precisions))
print("Recals:", recals, np.mean(recals), np.std(recals))

Epoch 1/50


 1/25 ━━━━━━━━━━━━━━━━━━━━ 27s 1s/step - AUC: 0.4416 - accuracy: 0.5000 - loss: 0.6916 - precision: 0.3333 - recall: 0.4545

23/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - AUC: 0.5650 - accuracy: 0.5654 - loss: 0.6864 - precision: 0.4542 - recall: 0.4991

25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - AUC: 0.5769 - accuracy: 0.5727 - loss: 0.6840 - precision: 0.4625 - recall: 0.5070 - val_AUC: 0.7998 - val_accuracy: 0.7500 - val_loss: 0.6087 - val_precision: 0.6421 - val_recall: 0.8133


Epoch 2/50


...

 1/25 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - AUC: 0.9529 - accuracy: 0.8750 - loss: 0.3203 - precision: 0.9231 - recall: 0.8000

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - AUC: 0.9491 - accuracy: 0.8911 - loss: 0.2851 - precision: 0.8434 - recall: 0.8938 - val_AUC: 0.7638 - val_accuracy: 0.6891 - val_loss: 0.7696 - val_precision: 0.5952 - val_recall: 0.6579


1/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - AUC: 0.8086 - accuracy: 0.7188 - loss: 0.5341 - precision: 0.7059 - recall: 0.7500

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - AUC: 0.8115 - accuracy: 0.7113 - loss: 0.5433 - precision: 0.6251 - recall: 0.7597


Loses: [0.5761836767196655, 0.610771119594574, 0.5097786784172058, 0.6079216003417969, 0.5684769153594971] 0.5746263980865478 0.036497888670375817
AUCs: [0.7728248834609985, 0.7855333089828491, 0.8284787535667419, 0.7475847005844116, 0.788896918296814] 0.784663712978363 0.026280962884538663
Accuracies: [0.7272727489471436, 0.6970954537391663, 0.7510373592376709, 0.6721991896629333, 0.7095435857772827] 0.7114296674728393 0.026712655630834647
Precisions: [0.6355140209197998, 0.5789473652839661, 0.6521739363670349, 0.5740740895271301, 0.605042040348053] 0.6091502904891968 0.030677401033807856
Recals: [0.7157894968986511, 0.8191489577293396, 0.7894737124443054, 0.6526315808296204, 0.75789475440979] 0.7469877004623413 0.058322683936944925
